In [73]:
import tkinter as tk
from tkinter import messagebox

In [76]:
root = tk.Tk()

HEIGHT = 450
WIDTH = 650

happy = 0.3
sad = 0.2
neutral = 0.1
angry = 0.01
disgusted = 0.08
fear = 0.2
Suprist = 0.2
buttonText = "auto"

def autoActivated():
    buttonText = "nogetAndet"
    tkMessageBox.showinfo( "Hello Python", "Hello World")


mood = "test1"
webCamRoute = "/Users/thomasmortensen/Desktop/GUITestPic.jpeg"
canvas = tk.Canvas(root, height = HEIGHT, width = WIDTH)
canvas.pack()

button = tk.Button(root, text = buttonText, command = autoActivated)
button.pack()


blueFrame = tk.Frame(root, bg = "blue")
blueFrame.place(rely = 0.1, relwidth = 0.5, relheight = 0.8)

webCamLabel = tk.Label(blueFrame, text="WebCam her", bg="gray")
webCamLabel.place(relx = 0.5, rely = 0.5, anchor = "center")
webCamLabel.pack()

redFrame = tk.Frame(root, bg = "red")
redFrame.place(relx = 0.5, rely = 0.1, relwidth = 0.5, relheight = 0.4)


moodLabel = tk.Label(redFrame, text = mood, bg = "red")
moodLabel.place(relx = 0.5, rely = 0.5, anchor = "center")

yellowFrame = tk.Frame(root, bg = "yellow")
yellowFrame.place(relx = 0.25, rely = 0.5, relwidth = 0.75, relheight = 0.4)

happyFrame = tk.Frame(yellowFrame, bg = "blue")
happyFrame.place(relx = 0.22, rely = 1, relwidth = 0.1, relheight = (0.9 * happy), anchor = "s")

sadFrame = tk.Frame(yellowFrame, bg = "red")
sadFrame.place(relx = 0.33, rely = 1, relwidth = 0.1, relheight = (0.9 * sad), anchor = "s")

neutralFrame = tk.Frame(yellowFrame, bg = "blue")
neutralFrame.place(relx = 0.44, rely = 1, relwidth = 0.1, relheight = (0.9 * neutral), anchor = "s")

angryFrame = tk.Frame(yellowFrame, bg = "red")
angryFrame.place(relx = 0.55, rely = 1, relwidth = 0.1, relheight = (0.9 * angry), anchor = "s")

xFrame = tk.Frame(yellowFrame, bg = "blue")
xFrame.place(relx = 0.66, rely = 1, relwidth = 0.1, relheight = (0.9 * x), anchor = "s")

yFrame = tk.Frame(yellowFrame, bg = "red")
yFrame.place(relx = 0.77, rely = 1, relwidth = 0.1, relheight = (0.9 * y), anchor = "s")

zFrame = tk.Frame(yellowFrame, bg = "blue")
zFrame.place(relx = 0.88, rely = 1, relwidth = 0.1, relheight = (0.9 * z), anchor ="s")




statsLabel = tk.Label(yellowFrame, text = "stats her")
statsLabel.pack()


root.mainloop()